In [3]:
qs = (
    BookingsFinancialData.objects.filter(**filters)
    .exclude(
        Q(credential__client__region__name__icontains="veci")
        | Q(credential__enabled=False)
    )
    .values()
)

levels_rules__ps__t__gt
levels_rules__mel__t__gt
levels_rules__cid__t__gt
levels_rules__bod__t__gt
levels_rules__dow__t__gt
levels_rules__room__t__gt
levels_rules__non__t__gt
levels_rules__hou__t__gt
levels_rules__mrk__t__gt
levels_rules__cp__gt
levels_rules__rat__gt
levels_rules__rel__gt
levels_rules__age__gt
